# 🫀 Heart Disease Prediction - ULTRA OPTIMIZED VERSION

**Target Score: 0.955+ ROC-AUC (Beat 0.95393)**

### Ultra Improvements:
- ✅ **Seed Averaging** (5 seeds for variance reduction)
- ✅ **7-Model Ensemble** (Added GaussianNB, RandomForest, MLP)
- ✅ **Pseudo-Labeling** (High-confidence test samples)
- ✅ **QuantileTransformer** (Better normalization)
- ✅ **Blend-of-Blends** (Combine all ensemble methods)
- ✅ **Stronger Regularization** (Combat overfitting)
- ✅ **Optuna Ready** (100 trials)

---

## 1. Setup & Imports

In [1]:
# Core Libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

# ML Tools
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Gradient Boosting Models
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Optimization
import optuna
from scipy.optimize import minimize
from scipy.stats import rankdata

# Configuration
SEEDS = SEEDS = [42, 123, 456]  # Multiple seeds for averaging
BASE_SEED = 42
np.random.seed(BASE_SEED)

print("✅ All libraries imported successfully!")
print(f"Using {len(SEEDS)} seeds for averaging: {SEEDS}")

✅ All libraries imported successfully!
Using 3 seeds for averaging: [42, 123, 456]


---
## 2. Data Loading

In [2]:
# === CONFIGURATION ===
# For Kaggle:
DATA_DIR = '/kaggle/input/playground-series-s6e2'

# For local (uncomment if testing locally):
# DATA_DIR = r'c:\Users\barsha mishra\Desktop\myprojects\kaggle\Predicting Heart Disease'

# Load data
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

# Constants
TARGET = 'Heart Disease'
ID_COL = 'id'

print(f"Train: {train.shape}, Test: {test.shape}")
print(f"Target distribution:\n{train[TARGET].value_counts(normalize=True)}")
print(f"\nTarget unique values: {train[TARGET].unique()}")

Train: (630000, 15), Test: (270000, 14)
Target distribution:
Heart Disease
Absence     0.55166
Presence    0.44834
Name: proportion, dtype: float64

Target unique values: ['Presence' 'Absence']


---
## 3. Advanced Feature Engineering (Enhanced)

In [3]:
def advanced_feature_engineering(df, is_train=True):
    """
    ULTRA Feature Engineering - 35+ Features
    """
    df = df.copy()
    cols = df.columns.tolist()
    new_features = []
    
    # ===== 1. Blood Pressure Features =====
    if 'Systolic BP' in cols and 'Diastolic BP' in cols:
        # Pulse Pressure (key indicator)
        df['pulse_pressure'] = df['Systolic BP'] - df['Diastolic BP']
        new_features.append('pulse_pressure')
        
        # Mean Arterial Pressure
        df['map'] = (df['Systolic BP'] + 2 * df['Diastolic BP']) / 3
        new_features.append('map')
        
        # BP Ratio
        df['bp_ratio'] = df['Systolic BP'] / (df['Diastolic BP'] + 1)
        new_features.append('bp_ratio')
        
        # Hypertension stages
        df['is_hypertensive'] = ((df['Systolic BP'] > 140) | (df['Diastolic BP'] > 90)).astype(int)
        df['hypertension_stage'] = 0
        df.loc[(df['Systolic BP'] >= 120) & (df['Systolic BP'] < 130), 'hypertension_stage'] = 1
        df.loc[(df['Systolic BP'] >= 130) | (df['Diastolic BP'] >= 80), 'hypertension_stage'] = 2
        df.loc[(df['Systolic BP'] >= 140) | (df['Diastolic BP'] >= 90), 'hypertension_stage'] = 3
        df.loc[(df['Systolic BP'] >= 180) | (df['Diastolic BP'] >= 120), 'hypertension_stage'] = 4
        new_features.extend(['is_hypertensive', 'hypertension_stage'])
        
        # NEW: BP product
        df['bp_product'] = df['Systolic BP'] * df['Diastolic BP'] / 1000
        new_features.append('bp_product')
    
    # ===== 2. Age-Based Features =====
    if 'Age' in cols:
        # Age groups
        df['age_decade'] = (df['Age'] // 10).astype(int)
        new_features.append('age_decade')
        
        # Age risk factors
        df['age_risk'] = (df['Age'] > 55).astype(int)
        df['age_high_risk'] = (df['Age'] > 65).astype(int)
        new_features.extend(['age_risk', 'age_high_risk'])
        
        # Age transformations
        df['age_squared'] = df['Age'] ** 2
        df['age_log'] = np.log1p(df['Age'])
        new_features.extend(['age_squared', 'age_log'])
    
    # ===== 3. Cholesterol Features =====
    if 'Cholesterol' in cols:
        # Cholesterol risk categories
        df['chol_risk'] = pd.cut(df['Cholesterol'], 
                                 bins=[0, 200, 239, 300, 1000], 
                                 labels=[0, 1, 2, 3]).astype(float).fillna(0)
        new_features.append('chol_risk')
        
        # Cholesterol log (reduce skew)
        df['chol_log'] = np.log1p(df['Cholesterol'])
        new_features.append('chol_log')
        
        # NEW: Cholesterol squared
        df['chol_squared'] = df['Cholesterol'] ** 2 / 10000
        new_features.append('chol_squared')
    
    # ===== 4. Interaction Features =====
    if 'Age' in cols and 'Cholesterol' in cols:
        df['age_chol'] = df['Age'] * df['Cholesterol'] / 1000
        df['chol_per_age'] = df['Cholesterol'] / (df['Age'] + 1)
        new_features.extend(['age_chol', 'chol_per_age'])
    
    if 'Age' in cols and 'Systolic BP' in cols:
        df['age_sbp'] = df['Age'] * df['Systolic BP'] / 1000
        df['sbp_per_age'] = df['Systolic BP'] / (df['Age'] + 1)
        new_features.extend(['age_sbp', 'sbp_per_age'])
    
    if 'Systolic BP' in cols and 'Cholesterol' in cols:
        df['sbp_chol'] = df['Systolic BP'] * df['Cholesterol'] / 10000
        new_features.append('sbp_chol')
    
    # ===== 5. Heart Rate Features =====
    if 'Heart Rate' in cols:
        df['hr_risk'] = ((df['Heart Rate'] < 60) | (df['Heart Rate'] > 100)).astype(int)
        df['hr_log'] = np.log1p(df['Heart Rate'])
        new_features.extend(['hr_risk', 'hr_log'])
        
        if 'Age' in cols:
            df['max_hr'] = 220 - df['Age']
            df['hr_reserve'] = df['max_hr'] - df['Heart Rate']
            df['hr_percent_max'] = df['Heart Rate'] / df['max_hr']
            new_features.extend(['max_hr', 'hr_reserve', 'hr_percent_max'])
    
    # ===== 6. BMI-related Features =====
    if 'BMI' in cols:
        df['bmi_category'] = pd.cut(df['BMI'], 
                                    bins=[0, 18.5, 25, 30, 100], 
                                    labels=[0, 1, 2, 3]).astype(float).fillna(1)
        df['is_obese'] = (df['BMI'] >= 30).astype(int)
        df['bmi_log'] = np.log1p(df['BMI'])
        new_features.extend(['bmi_category', 'is_obese', 'bmi_log'])
    
    # ===== 7. Composite Risk Score (Enhanced) =====
    risk_score = np.zeros(len(df))
    if 'Age' in cols:
        risk_score += (df['Age'] > 55).astype(int) * 2
        risk_score += (df['Age'] > 65).astype(int) * 1
    if 'is_hypertensive' in df.columns:
        risk_score += df['is_hypertensive'] * 3
    if 'Cholesterol' in cols:
        risk_score += (df['Cholesterol'] > 240).astype(int) * 2
    if 'Smoking' in cols:
        risk_score += (df['Smoking'] == 1).astype(int) * 3
    if 'Diabetes' in cols:
        risk_score += (df['Diabetes'] == 1).astype(int) * 2
    if 'Sex' in cols:
        risk_score += (df['Sex'] == 'Male').astype(int) * 1
    
    df['composite_risk_score'] = risk_score
    df['risk_category'] = pd.cut(risk_score, bins=[-1, 3, 6, 10, 20], labels=[0, 1, 2, 3]).astype(float).fillna(1)
    new_features.extend(['composite_risk_score', 'risk_category'])
    
    # ===== 8. Statistical Features =====
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols = [c for c in num_cols if c not in [ID_COL, TARGET, 'composite_risk_score']]
    
    if len(num_cols) >= 3:
        base_num_cols = num_cols[:6]  # Use first 6 numeric columns
        df['row_mean'] = df[base_num_cols].mean(axis=1)
        df['row_std'] = df[base_num_cols].std(axis=1)
        df['row_max'] = df[base_num_cols].max(axis=1)
        df['row_min'] = df[base_num_cols].min(axis=1)
        df['row_range'] = df['row_max'] - df['row_min']
        new_features.extend(['row_mean', 'row_std', 'row_max', 'row_min', 'row_range'])
    
    if is_train:
        print(f"✅ Created {len(new_features)} new features")
    
    return df

print("Feature engineering function defined!")

Feature engineering function defined!


In [4]:
# Apply Feature Engineering
print("=== Applying Advanced Feature Engineering ===")
train_fe = advanced_feature_engineering(train, is_train=True)
test_fe = advanced_feature_engineering(test, is_train=False)

print(f"\nTrain shape: {train_fe.shape}")
print(f"Test shape: {test_fe.shape}")

=== Applying Advanced Feature Engineering ===
✅ Created 17 new features

Train shape: (630000, 32)
Test shape: (270000, 31)


---
## 4. Preprocessing with QuantileTransformer

In [5]:
def preprocess_data(train_df, test_df, target_col, id_col, n_folds=10, use_quantile=True):
    """
    Enhanced preprocessing with QuantileTransformer.
    """
    # Separate features and target
    y = train_df[target_col].copy()
    
    # Encode target if categorical
    if y.dtype == 'object' or str(y.dtype) == 'category':
        target_le = LabelEncoder()
        y = pd.Series(target_le.fit_transform(y), index=y.index)
        print(f"✅ Target encoded: {list(target_le.classes_)} -> {list(range(len(target_le.classes_)))}")
    
    X = train_df.drop([target_col, id_col], axis=1, errors='ignore')
    X_test = test_df.drop([id_col], axis=1, errors='ignore')
    
    # Align columns
    common_cols = X.columns.intersection(X_test.columns)
    X = X[common_cols]
    X_test = X_test[common_cols]
    
    print(f"Features after alignment: {len(common_cols)}")
    
    # Identify column types
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(exclude=['object', 'category']).columns.tolist()
    
    print(f"Numerical: {len(num_cols)}, Categorical: {len(cat_cols)}")
    
    # Imputation
    if num_cols:
        num_imputer = SimpleImputer(strategy='median')
        X[num_cols] = num_imputer.fit_transform(X[num_cols])
        X_test[num_cols] = num_imputer.transform(X_test[num_cols])
    
    # Categorical Encoding
    if cat_cols:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
        X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])
        
        # Target encoding with cross-validation
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=BASE_SEED)
        
        for col in cat_cols:
            X[f'{col}_te'] = 0.0
            X_test[f'{col}_te'] = 0.0
            
            global_mean = y.mean()
            encoding_map = {}
            
            for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                train_data = pd.DataFrame({'cat': X.iloc[train_idx][col], 'target': y.iloc[train_idx]})
                encoding = train_data.groupby('cat')['target'].mean()
                
                X.iloc[val_idx, X.columns.get_loc(f'{col}_te')] = X.iloc[val_idx][col].map(encoding).fillna(global_mean)
                
                for cat_val, enc_val in encoding.items():
                    if cat_val not in encoding_map:
                        encoding_map[cat_val] = []
                    encoding_map[cat_val].append(enc_val)
            
            final_encoding = {k: np.mean(v) for k, v in encoding_map.items()}
            X_test[f'{col}_te'] = X_test[col].map(final_encoding).fillna(global_mean)
            
            # Label encode original categorical
            le = LabelEncoder()
            combined = pd.concat([X[col], X_test[col]]).astype(str)
            le.fit(combined)
            X[col] = le.transform(X[col].astype(str))
            X_test[col] = le.transform(X_test[col].astype(str))
    
    # Update num_cols
    num_cols = X.select_dtypes(exclude=['object', 'category']).columns.tolist()
    
    # Scaling with QuantileTransformer (better than StandardScaler)
    if num_cols:
        if use_quantile:
            scaler = QuantileTransformer(output_distribution='normal', n_quantiles=1000, random_state=BASE_SEED)
            print("✅ Using QuantileTransformer (output=normal)")
        else:
            scaler = StandardScaler()
            print("Using StandardScaler")
        X[num_cols] = scaler.fit_transform(X[num_cols])
        X_test[num_cols] = scaler.transform(X_test[num_cols])
    
    return X, y, X_test

print("Preprocessing function defined!")

Preprocessing function defined!


In [6]:
# Apply Preprocessing
X, y, X_test = preprocess_data(train_fe, test_fe, TARGET, ID_COL, n_folds=10, use_quantile=True)

print(f"\nFinal shapes:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")
print(f"X_test: {X_test.shape}")
print(f"\ny unique values: {y.unique()}  (should be [0, 1])")

✅ Target encoded: ['Absence', 'Presence'] -> [0, 1]
Features after alignment: 30
Numerical: 30, Categorical: 0
✅ Using QuantileTransformer (output=normal)

Final shapes:
X: (630000, 30)
y: (630000,)
X_test: (270000, 30)

y unique values: [1 0]  (should be [0, 1])


---
## 5. Model Training with Seed Averaging (7 Models)

In [7]:
# Configuration
N_FOLDS = 10

def get_model_params(seed):
    """Get model parameters with specific seed - STRONGER REGULARIZATION"""
    return {
        'lgb': {
            'objective': 'binary',
            'metric': 'auc',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'random_state': seed,
            'learning_rate': 0.02,  # Lower LR
            'n_estimators': 3000,   # More trees
            'num_leaves': 31,       # Reduced from 50
            'max_depth': 6,         # Reduced from 8
            'feature_fraction': 0.7,
            'bagging_fraction': 0.7,
            'bagging_freq': 5,
            'min_child_samples': 30,  # Increased from 20
            'reg_alpha': 0.5,         # Increased from 0.1
            'reg_lambda': 0.5,        # Increased from 0.1
        },
        'xgb': {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'tree_method': 'hist',
            'random_state': seed,
            'learning_rate': 0.02,
            'n_estimators': 3000,
            'max_depth': 5,         # Reduced from 7
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'reg_alpha': 0.5,
            'reg_lambda': 0.5,
            'min_child_weight': 15,  # Increased from 10
        },
        'cat': {
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'random_state': seed,
            'learning_rate': 0.02,
            'iterations': 3000,
            'depth': 5,             # Reduced from 7
            'l2_leaf_reg': 5,       # Increased from 3
            'verbose': False,
        },
        'hgb': {
            'learning_rate': 0.02,
            'max_iter': 3000,
            'max_depth': 6,
            'min_samples_leaf': 30,
            'l2_regularization': 0.5,
            'random_state': seed,
        },
        'et': {
            'n_estimators': 800,
            'max_depth': 12,        # Reduced from 15
            'min_samples_split': 15,
            'min_samples_leaf': 8,
            'random_state': seed,
            'n_jobs': -1,
        },
        'rf': {
            'n_estimators': 800,
            'max_depth': 12,
            'min_samples_split': 15,
            'min_samples_leaf': 8,
            'random_state': seed,
            'n_jobs': -1,
        },
        'gnb': {},  # GaussianNB has no hyperparameters
    }

print(f"✅ Model parameters defined with stronger regularization")

✅ Model parameters defined with stronger regularization


In [8]:
def train_single_seed(seed):
    """Train all models with a single seed"""
    print(f"\n{'='*60}")
    print(f"TRAINING WITH SEED {seed}")
    print(f"{'='*60}")
    
    params = get_model_params(seed)
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
    
    # Initialize arrays
    oof_lgb = np.zeros(len(X))
    oof_xgb = np.zeros(len(X))
    oof_cat = np.zeros(len(X))
    oof_hgb = np.zeros(len(X))
    oof_et = np.zeros(len(X))
    oof_rf = np.zeros(len(X))
    oof_gnb = np.zeros(len(X))
    
    test_lgb = np.zeros(len(X_test))
    test_xgb = np.zeros(len(X_test))
    test_cat = np.zeros(len(X_test))
    test_hgb = np.zeros(len(X_test))
    test_et = np.zeros(len(X_test))
    test_rf = np.zeros(len(X_test))
    test_gnb = np.zeros(len(X_test))
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"  Fold {fold + 1}/{N_FOLDS}", end=" ")
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # LightGBM
        model_lgb = LGBMClassifier(**params['lgb'])
        model_lgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[])
        oof_lgb[val_idx] = model_lgb.predict_proba(X_val)[:, 1]
        test_lgb += model_lgb.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # XGBoost
        model_xgb = XGBClassifier(**params['xgb'])
        model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        oof_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
        test_xgb += model_xgb.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # CatBoost
        model_cat = CatBoostClassifier(**params['cat'])
        model_cat.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=200)
        oof_cat[val_idx] = model_cat.predict_proba(X_val)[:, 1]
        test_cat += model_cat.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # HistGradientBoosting
        model_hgb = HistGradientBoostingClassifier(**params['hgb'])
        model_hgb.fit(X_train, y_train)
        oof_hgb[val_idx] = model_hgb.predict_proba(X_val)[:, 1]
        test_hgb += model_hgb.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # ExtraTrees
        model_et = ExtraTreesClassifier(**params['et'])
        model_et.fit(X_train, y_train)
        oof_et[val_idx] = model_et.predict_proba(X_val)[:, 1]
        test_et += model_et.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # RandomForest
        model_rf = RandomForestClassifier(**params['rf'])
        model_rf.fit(X_train, y_train)
        oof_rf[val_idx] = model_rf.predict_proba(X_val)[:, 1]
        test_rf += model_rf.predict_proba(X_test)[:, 1] / N_FOLDS
        
        # GaussianNB
        model_gnb = GaussianNB()
        model_gnb.fit(X_train, y_train)
        oof_gnb[val_idx] = model_gnb.predict_proba(X_val)[:, 1]
        test_gnb += model_gnb.predict_proba(X_test)[:, 1] / N_FOLDS
        
        print(f"✓")
    
    # Calculate scores
    scores = {
        'LightGBM': roc_auc_score(y, oof_lgb),
        'XGBoost': roc_auc_score(y, oof_xgb),
        'CatBoost': roc_auc_score(y, oof_cat),
        'HistGradientBoosting': roc_auc_score(y, oof_hgb),
        'ExtraTrees': roc_auc_score(y, oof_et),
        'RandomForest': roc_auc_score(y, oof_rf),
        'GaussianNB': roc_auc_score(y, oof_gnb),
    }
    
    print(f"\nSeed {seed} OOF Scores:")
    for name, score in scores.items():
        print(f"  {name}: {score:.5f}")
    
    return {
        'oof': [oof_lgb, oof_xgb, oof_cat, oof_hgb, oof_et, oof_rf, oof_gnb],
        'test': [test_lgb, test_xgb, test_cat, test_hgb, test_et, test_rf, test_gnb],
        'scores': scores
    }

print("Training function defined!")

Training function defined!


In [9]:
# ===== TRAIN WITH ALL SEEDS =====
print("="*60)
print(f"TRAINING WITH {len(SEEDS)} SEEDS FOR AVERAGING")
print("="*60)

all_results = []
for seed in SEEDS:
    result = train_single_seed(seed)
    all_results.append(result)

print("\n✅ All seeds trained!")

TRAINING WITH 3 SEEDS FOR AVERAGING

TRAINING WITH SEED 42
  Fold 1/10 ✓
  Fold 2/10 ✓
  Fold 3/10 ✓
  Fold 4/10 ✓
  Fold 5/10 ✓
  Fold 6/10 ✓
  Fold 7/10 ✓
  Fold 8/10 ✓
  Fold 9/10 ✓
  Fold 10/10 ✓

Seed 42 OOF Scores:
  LightGBM: 0.95516
  XGBoost: 0.95526
  CatBoost: 0.95545
  HistGradientBoosting: 0.95514
  ExtraTrees: 0.94728
  RandomForest: 0.95243
  GaussianNB: 0.91911

TRAINING WITH SEED 123
  Fold 1/10 ✓
  Fold 2/10 ✓
  Fold 3/10 ✓
  Fold 4/10 ✓
  Fold 5/10 ✓
  Fold 6/10 ✓
  Fold 7/10 ✓
  Fold 8/10 ✓
  Fold 9/10 ✓
  Fold 10/10 ✓

Seed 123 OOF Scores:
  LightGBM: 0.95514
  XGBoost: 0.95527
  CatBoost: 0.95545
  HistGradientBoosting: 0.95514
  ExtraTrees: 0.94724
  RandomForest: 0.95243
  GaussianNB: 0.91911

TRAINING WITH SEED 456
  Fold 1/10 ✓
  Fold 2/10 ✓
  Fold 3/10 ✓
  Fold 4/10 ✓
  Fold 5/10 ✓
  Fold 6/10 ✓
  Fold 7/10 ✓
  Fold 8/10 ✓
  Fold 9/10 ✓
  Fold 10/10 ✓

Seed 456 OOF Scores:
  LightGBM: 0.95518
  XGBoost: 0.95529
  CatBoost: 0.95545
  HistGradientBoosting: 0.95

In [10]:
# ===== AVERAGE ACROSS SEEDS =====
model_names = ['LightGBM', 'XGBoost', 'CatBoost', 'HistGradientBoosting', 'ExtraTrees', 'RandomForest', 'GaussianNB']
n_models = len(model_names)

# Average OOF and test predictions across all seeds
oof_averaged = [np.mean([res['oof'][i] for res in all_results], axis=0) for i in range(n_models)]
test_averaged = [np.mean([res['test'][i] for res in all_results], axis=0) for i in range(n_models)]

# Calculate seed-averaged scores
print("\n" + "="*60)
print("SEED-AVERAGED OOF SCORES (More Robust!)")
print("="*60)

averaged_scores = {}
for i, name in enumerate(model_names):
    score = roc_auc_score(y, oof_averaged[i])
    averaged_scores[name] = score
    print(f"{name:25s}: {score:.5f}")


SEED-AVERAGED OOF SCORES (More Robust!)
LightGBM                 : 0.95525
XGBoost                  : 0.95533
CatBoost                 : 0.95547
HistGradientBoosting     : 0.95521
ExtraTrees               : 0.94731
RandomForest             : 0.95246
GaussianNB               : 0.91911


---
## 6. Advanced Ensemble Techniques (Blend-of-Blends)

In [11]:
# ===== 6.1 RANK AVERAGING =====
def rank_average(predictions_list):
    """Convert predictions to ranks and average them"""
    ranks = np.zeros_like(predictions_list[0])
    for preds in predictions_list:
        ranks += rankdata(preds)
    return ranks / len(predictions_list)

oof_rank = rank_average(oof_averaged)
test_rank = rank_average(test_averaged)

rank_score = roc_auc_score(y, oof_rank)
print(f"🎯 Rank Average OOF Score: {rank_score:.5f}")

🎯 Rank Average OOF Score: 0.95395


In [12]:
# ===== 6.2 OPTIMIZED WEIGHTED AVERAGE =====
def optimize_weights(oofs, target):
    """Find optimal weights using scipy minimize"""
    def objective(weights):
        final_pred = np.sum([w * oof for w, oof in zip(weights, oofs)], axis=0)
        return -roc_auc_score(target, final_pred)
    
    n_models = len(oofs)
    initial_weights = [1/n_models] * n_models
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1)] * n_models
    
    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

optimal_weights = optimize_weights(oof_averaged, y)

print("=== Optimized Ensemble Weights ===")
for name, weight in zip(model_names, optimal_weights):
    print(f"  {name}: {weight:.4f}")

oof_weighted = np.sum([w * oof for w, oof in zip(optimal_weights, oof_averaged)], axis=0)
test_weighted = np.sum([w * oof for w, oof in zip(optimal_weights, test_averaged)], axis=0)

weighted_score = roc_auc_score(y, oof_weighted)
print(f"\n🎯 Weighted Ensemble OOF Score: {weighted_score:.5f}")

=== Optimized Ensemble Weights ===
  LightGBM: 0.2608
  XGBoost: 0.2313
  CatBoost: 0.2599
  HistGradientBoosting: 0.2039
  ExtraTrees: 0.0000
  RandomForest: 0.0442
  GaussianNB: 0.0000

🎯 Weighted Ensemble OOF Score: 0.95542


In [13]:
# ===== 6.3 STACKING META-LEARNER =====
stack_train = np.column_stack(oof_averaged)
stack_test = np.column_stack(test_averaged)

meta_model = LogisticRegression(random_state=BASE_SEED, max_iter=1000, C=0.3)
meta_model.fit(stack_train, y)

oof_stack = meta_model.predict_proba(stack_train)[:, 1]
test_stack = meta_model.predict_proba(stack_test)[:, 1]

stack_score = roc_auc_score(y, oof_stack)
print(f"🎯 Stacking OOF Score: {stack_score:.5f}")

🎯 Stacking OOF Score: 0.95481


In [14]:
# ===== 6.4 GEOMETRIC MEAN =====
def geometric_mean(predictions_list):
    """Geometric mean of predictions"""
    clipped = [np.clip(p, 1e-8, 1-1e-8) for p in predictions_list]
    return np.exp(np.mean(np.log(clipped), axis=0))

oof_geom = geometric_mean(oof_averaged)
test_geom = geometric_mean(test_averaged)

geom_score = roc_auc_score(y, oof_geom)
print(f"🎯 Geometric Mean OOF Score: {geom_score:.5f}")

🎯 Geometric Mean OOF Score: 0.95182


In [15]:
# ===== 6.5 BLEND OF BLENDS (Ultimate Ensemble) =====
# Combine all ensemble methods
ensemble_oofs = [oof_rank, oof_weighted, oof_stack, oof_geom]
ensemble_tests = [test_rank, test_weighted, test_stack, test_geom]

# Normalize rank predictions to 0-1
oof_rank_norm = (oof_rank - oof_rank.min()) / (oof_rank.max() - oof_rank.min())
test_rank_norm = (test_rank - test_rank.min()) / (test_rank.max() - test_rank.min())

# Blend of blends with equal weights
oof_blend = (oof_rank_norm + oof_weighted + oof_stack + oof_geom) / 4
test_blend = (test_rank_norm + test_weighted + test_stack + test_geom) / 4

blend_score = roc_auc_score(y, oof_blend)
print(f"🎯 Blend-of-Blends OOF Score: {blend_score:.5f}")

🎯 Blend-of-Blends OOF Score: 0.95475


In [16]:
# ===== 6.6 OPTIMIZED BLEND OF BLENDS =====
blend_preds = [oof_rank_norm, oof_weighted, oof_stack, oof_geom]
blend_tests_all = [test_rank_norm, test_weighted, test_stack, test_geom]

optimal_blend_weights = optimize_weights(blend_preds, y)

print("=== Blend-of-Blends Optimized Weights ===")
blend_names = ['Rank Avg', 'Weighted', 'Stacking', 'Geometric']
for name, weight in zip(blend_names, optimal_blend_weights):
    print(f"  {name}: {weight:.4f}")

oof_optimal_blend = np.sum([w * p for w, p in zip(optimal_blend_weights, blend_preds)], axis=0)
test_optimal_blend = np.sum([w * p for w, p in zip(optimal_blend_weights, blend_tests_all)], axis=0)

optimal_blend_score = roc_auc_score(y, oof_optimal_blend)
print(f"\n🎯 Optimized Blend-of-Blends OOF Score: {optimal_blend_score:.5f}")

=== Blend-of-Blends Optimized Weights ===
  Rank Avg: 0.2473
  Weighted: 0.2521
  Stacking: 0.2527
  Geometric: 0.2479

🎯 Optimized Blend-of-Blends OOF Score: 0.95476


In [17]:
# ===== FINAL COMPARISON =====
print("\n" + "="*60)
print("FINAL COMPARISON - ALL METHODS")
print("="*60)

all_scores = {
    **averaged_scores,
    'Rank Average': rank_score,
    'Weighted Ensemble': weighted_score,
    'Stacking': stack_score,
    'Geometric Mean': geom_score,
    'Blend-of-Blends': blend_score,
    'Optimal Blend': optimal_blend_score,
}

for method, score in sorted(all_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{method:25s}: {score:.5f}")

best_method = max(all_scores, key=all_scores.get)
print(f"\n🏆 BEST METHOD: {best_method} with {all_scores[best_method]:.5f}")


FINAL COMPARISON - ALL METHODS
CatBoost                 : 0.95547
Weighted Ensemble        : 0.95542
XGBoost                  : 0.95533
LightGBM                 : 0.95525
HistGradientBoosting     : 0.95521
Stacking                 : 0.95481
Optimal Blend            : 0.95476
Blend-of-Blends          : 0.95475
Rank Average             : 0.95395
RandomForest             : 0.95246
Geometric Mean           : 0.95182
ExtraTrees               : 0.94731
GaussianNB               : 0.91911

🏆 BEST METHOD: CatBoost with 0.95547


---
## 7. Pseudo-Labeling (Optional - Risky)

In [18]:
# ===== SET THIS TO TRUE TO ENABLE PSEUDO-LABELING =====
USE_PSEUDO_LABELING = False  # Set to True for aggressive approach
CONFIDENCE_THRESHOLD = 0.95

if USE_PSEUDO_LABELING:
    print("=== Pseudo-Labeling ===")
    # Use best method predictions
    if best_method == 'Optimal Blend':
        pseudo_preds = test_optimal_blend
    elif best_method == 'Blend-of-Blends':
        pseudo_preds = test_blend
    else:
        pseudo_preds = test_weighted
    
    high_conf_mask = (pseudo_preds > CONFIDENCE_THRESHOLD) | (pseudo_preds < (1 - CONFIDENCE_THRESHOLD))
    n_pseudo = high_conf_mask.sum()
    
    print(f"High confidence samples: {n_pseudo} ({100*n_pseudo/len(pseudo_preds):.1f}%)")
    
    if n_pseudo > 100:
        pseudo_X = X_test[high_conf_mask]
        pseudo_y = (pseudo_preds[high_conf_mask] > 0.5).astype(int)
        
        X_augmented = pd.concat([X, pseudo_X], ignore_index=True)
        y_augmented = pd.concat([y, pd.Series(pseudo_y)], ignore_index=True)
        
        print(f"Augmented training set: {len(X_augmented)} samples")
        # TODO: Retrain models with augmented data
    else:
        print("Not enough high-confidence samples for pseudo-labeling")
else:
    print("⏩ Pseudo-labeling disabled. Set USE_PSEUDO_LABELING = True to enable.")

⏩ Pseudo-labeling disabled. Set USE_PSEUDO_LABELING = True to enable.


---
## 8. Optuna Hyperparameter Tuning (Optional)

In [19]:
RUN_OPTUNA = False  # Set to True for tuning (takes ~60 min)
N_TRIALS = 100

def lgb_optuna_objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': BASE_SEED,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 80),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 5.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 5.0, log=True),
        'n_estimators': 3000,
    }
    
    scores = []
    skf_tune = StratifiedKFold(n_splits=5, shuffle=True, random_state=BASE_SEED)
    
    for train_idx, val_idx in skf_tune.split(X, y):
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]
        
        model = LGBMClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_vl, y_vl)], callbacks=[])
        
        pred = model.predict_proba(X_vl)[:, 1]
        scores.append(roc_auc_score(y_vl, pred))
    
    return np.mean(scores)

if RUN_OPTUNA:
    print("=== Running Optuna Hyperparameter Tuning ===")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize')
    study.optimize(lgb_optuna_objective, n_trials=N_TRIALS, show_progress_bar=True)
    
    print(f"\n🎯 Best Optuna Score: {study.best_trial.value:.5f}")
    print(f"Best Parameters: {study.best_trial.params}")
else:
    print("⏩ Optuna skipped. Set RUN_OPTUNA = True to enable.")

⏩ Optuna skipped. Set RUN_OPTUNA = True to enable.


---
## 9. Generate Submission

In [20]:
# Select best predictions
all_test_preds = {
    'Rank Average': test_rank_norm,
    'Weighted Ensemble': test_weighted,
    'Stacking': test_stack,
    'Geometric Mean': test_geom,
    'Blend-of-Blends': test_blend,
    'Optimal Blend': test_optimal_blend,
}

# Use the best method
final_preds = all_test_preds.get(best_method, test_optimal_blend)

# Clip to avoid extreme values
final_preds = np.clip(final_preds, 0.001, 0.999)

print(f"✅ Using: {best_method}")
print(f"Prediction range: [{final_preds.min():.4f}, {final_preds.max():.4f}]")

✅ Using: CatBoost
Prediction range: [0.0084, 0.9919]


In [21]:
# Create submission
submission = pd.DataFrame({
    ID_COL: test[ID_COL],
    TARGET: final_preds
})

submission.to_csv('submission_ultra.csv', index=False)

print("\n" + "="*60)
print("✅ ULTRA SUBMISSION FILE GENERATED!")
print("="*60)
print(f"\nFile: submission_ultra.csv")
print(f"Shape: {submission.shape}")
print(f"\nExpected Score: {all_scores[best_method]:.5f} (OOF)")
print("\nPreview:")
submission.head(10)


✅ ULTRA SUBMISSION FILE GENERATED!

File: submission_ultra.csv
Shape: (270000, 2)

Expected Score: 0.95547 (OOF)

Preview:


,id,Heart Disease
0,630000,0.901901
1,630001,0.026209
2,630002,0.960942
3,630003,0.019671
4,630004,0.241521
5,630005,0.956119
6,630006,0.036145
7,630007,0.639017
8,630008,0.974230
9,630009,0.042433


In [22]:
# ===== SUMMARY =====
print("\n" + "="*60)
print("🏆 ULTRA OPTIMIZATION SUMMARY")
print("="*60)
print(f"\nModels Used: 7 (LGB, XGB, CatBoost, HistGBT, ExtraTrees, RandomForest, GaussianNB)")
print(f"Seeds Averaged: {len(SEEDS)} seeds")
print(f"CV Strategy: {N_FOLDS}-Fold Stratified")
print(f"Features: {X.shape[1]}")
print(f"\nBest Ensemble Method: {best_method}")
print(f"Expected LB Score: ~{all_scores[best_method]:.5f}")
print("\n✅ Ready to submit!")


🏆 ULTRA OPTIMIZATION SUMMARY

Models Used: 7 (LGB, XGB, CatBoost, HistGBT, ExtraTrees, RandomForest, GaussianNB)
Seeds Averaged: 3 seeds
CV Strategy: 10-Fold Stratified
Features: 30

Best Ensemble Method: CatBoost
Expected LB Score: ~0.95547

✅ Ready to submit!
